In [2]:
from influxdb_client import InfluxDBClient
import pandas as pd

client = InfluxDBClient(url='{url}', token='{token}', org='{org}')

query = """
from(bucket: "netrics-prod0")
  |> range(start: {}, stop: {})
  |> filter(fn: (r) => r["_measurement"] == "ookla")
  |> filter(fn: (r) => r["_field"] == "speedtest_ookla_download" or r["_field"] == "speedtest_ookla_upload")
"""

fquery = query.format('2021-10-01', '2022-06-30')

result = client.query_api().query_data_frame(fquery)

In [3]:
result.head()

,result,table,_start,_stop,_time,_value,_field,_measurement,install,meta_extended_dataver,meta_extended_debhash
0,_result,0,2021-10-01 00:00:00+00:00,2022-06-30 00:00:00+00:00,2021-10-08 22:12:21.988136+00:00,475.609224,speedtest_ookla_download,ookla,nm-mngd-20210317-3b1681ec,0,EBHASH}
1,_result,0,2021-10-01 00:00:00+00:00,2022-06-30 00:00:00+00:00,2021-10-08 22:19:17.489398+00:00,233.039768,speedtest_ookla_download,ookla,nm-mngd-20210317-3b1681ec,0,EBHASH}
2,_result,1,2021-10-01 00:00:00+00:00,2022-06-30 00:00:00+00:00,2021-10-08 22:12:21.988136+00:00,23.933456,speedtest_ookla_upload,ookla,nm-mngd-20210317-3b1681ec,0,EBHASH}
3,_result,1,2021-10-01 00:00:00+00:00,2022-06-30 00:00:00+00:00,2021-10-08 22:19:17.489398+00:00,23.866664,speedtest_ookla_upload,ookla,nm-mngd-20210317-3b1681ec,0,EBHASH}
4,_result,2,2021-10-01 00:00:00+00:00,2022-06-30 00:00:00+00:00,2021-10-01 22:06:37.553326+00:00,476.289808,speedtest_ookla_download,ookla,nm-mngd-20210317-3b1681ec,1,223dc2a


In [21]:
resampled = result.set_index('_time').groupby('install').resample('1M').count()

In [22]:
resampled = resampled.drop('install', axis=1).reset_index()

In [26]:
filtered = resampled[resampled._value > 20]

In [30]:
filtered['device_on'] = 1
filtered.groupby('_time').device_on.count()

<ipython-input-30-959af1a79373>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['device_on'] = 1


_time
2021-10-31 00:00:00+00:00    55
2021-11-30 00:00:00+00:00    57
2021-12-31 00:00:00+00:00    49
2022-01-31 00:00:00+00:00    51
2022-02-28 00:00:00+00:00    55
2022-03-31 00:00:00+00:00    70
2022-04-30 00:00:00+00:00    62
2022-05-31 00:00:00+00:00    85
2022-06-30 00:00:00+00:00    88
Name: device_on, dtype: int64

In [32]:
filtered.groupby('_time').install.nunique()

_time
2021-10-31 00:00:00+00:00    55
2021-11-30 00:00:00+00:00    57
2021-12-31 00:00:00+00:00    49
2022-01-31 00:00:00+00:00    51
2022-02-28 00:00:00+00:00    55
2022-03-31 00:00:00+00:00    70
2022-04-30 00:00:00+00:00    62
2022-05-31 00:00:00+00:00    85
2022-06-30 00:00:00+00:00    88
Name: install, dtype: int64